# Data loading and normalizing

In [1]:
import io

poems = list(line[1:-1] for line in 
             io.open('../datasets/Shahnameh-Poems.txt', mode="r", encoding="utf-8").readlines())[:1_000]

import pandas as pd
import numpy as np
import hazm

poems = np.array(poems)
poems = np.apply_along_axis(' '.join, 1, poems.reshape(-1, 2))

df = pd.DataFrame(poems, columns=['poems'])

normalizer = hazm.Normalizer(token_based=True)

df['normal'] = df.poems.apply(normalizer.normalize)

# Tfidf and Boolean approaches

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('count', CountVectorizer(analyzer='word', ngram_range=(1, 1), max_features=20_000)),
                 ('tfidf', TfidfTransformer(sublinear_tf=True))]).fit(df.normal)

def get_document_vectors(series):
    boolw_vec = pipe['count'].transform(series).toarray().astype(bool).astype(int)
    norm = np.linalg.norm(boolw_vec, axis=1).reshape(-1, 1)
    return pipe.transform(series).toarray() , boolw_vec / norm

tfidf_documents, boolw_documents = get_document_vectors(df.normal)

def get_similars_by_cosine_distance(vector, documents, n=5):
    sq_vector = np.squeeze(vector)
    similarity = documents.dot(sq_vector) / (np.linalg.norm(documents, axis=1) * np.linalg.norm(sq_vector))
    sorted_indx = np.argsort(similarity)
    
    return list(zip(df.poems[sorted_indx[-n:]], similarity[sorted_indx[-n:]]))

tfidf_vector, boolw_vector = \
    get_document_vectors(['رستم زد توی سر سیاوش و به گردآفرید گفت برو با بزرگترت بیا'])

print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(tfidf_vector, tfidf_documents):
    print("tfidf: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(boolw_vector, boolw_documents):
    print("boolw: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

----------------------------------------------------------------------------------------------------
tfidf: "بدو گفت من چاره سازم ترا به خورشید سر برفرازم تر" 	 with similarity of 0.20
tfidf: "جهاندار هوشنگ با رای و داد به جای نیا تاج بر سر " 	 with similarity of 0.20
tfidf: "برو تاختن کرد ناگاه مرگ نهادش به سر بر یکی تیره " 	 with similarity of 0.24
tfidf: "نشسته برو شهریاری چو ماه یکی تاج بر سر به جای کل" 	 with similarity of 0.24
tfidf: "به رشک اندر آهرمن بدسگال همی رای زد تا ببالید با" 	 with similarity of 0.33
----------------------------------------------------------------------------------------------------
boolw: "برو تاختن کرد ناگاه مرگ نهادش به سر بر یکی تیره " 	 with similarity of 0.37
boolw: "به رشک اندر آهرمن بدسگال همی رای زد تا ببالید با" 	 with similarity of 0.37
boolw: "جهاندار هوشنگ با رای و داد به جای نیا تاج بر سر " 	 with similarity of 0.37
boolw: "سر بخت و تختش برآمد به کوه پلنگینه پوشید خود با " 	 with similarity of 0.39
boolw: "سر و تن بشستی نهفته به باغ پرستنده

In [3]:
del tfidf_documents
del boolw_documents

word_idfs = dict(zip(pipe['count'].get_feature_names_out(), pipe['tfidf'].idf_))

del pipe
del tfidf_vector
del boolw_vector

import gc
gc.collect()

0

# Combining word embedding and idf

In [4]:
from gensim.models import KeyedVectors
from hazm import word_tokenize

word2vec = KeyedVectors.load_word2vec_format('../models/farsi_literature_word2vec_model.txt')

def embed(poem):
    embedding_vectors = [word2vec[wo] * word_idfs.get(wo, 0) for wo in word_tokenize(poem) if wo in word2vec]
    return np.sum(embedding_vectors, axis=0).tolist()

poems_embeddings = np.array(df.normal.apply(embed).tolist())

sample_embedding = np.array(embed('رستم زد توی سر سیاوش و به گردافرید گفت برو با بزرگترت بیا'))

print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(sample_embedding, poems_embeddings):
    print("embedding: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

----------------------------------------------------------------------------------------------------
embedding: "چو بنشست بر جایگاه مهی چنین گفت بر تخت شاهنشهی" 	 with similarity of 0.41
embedding: "برفت اهرمن را به افسون ببست چو بر تیزرو بارگی بر" 	 with similarity of 0.42
embedding: "برنجید و گسترد و خورد و سپرد برفت و به جز نام نی" 	 with similarity of 0.46
embedding: "به رشک اندر آهرمن بدسگال همی رای زد تا ببالید با" 	 with similarity of 0.48
embedding: "پژوهندهٔ نامهٔ باستان که از پهلوانان زند داستان" 	 with similarity of 0.48


In [5]:
del word2vec
del poems_embeddings
del sample_embedding

import gc
gc.collect()

0

# Use BigBird and ParsBert last hidden state as embeddings

In [6]:
from transformers import BigBirdModel, AutoTokenizer, AutoConfig, AutoModel

MODEL_NAME = "SajjadAyoubi/distil-bigbird-fa-zwnj"

model = BigBirdModel.from_pretrained(MODEL_NAME, attention_type="original_full")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

2022-06-04 00:53:25.927552: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/alireza/.mujoco/mujoco210/bin
2022-06-04 00:53:25.927574: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Some weights of the model checkpoint at SajjadAyoubi/distil-bigbird-fa-zwnj were not used when initializing BigBirdModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequ

In [7]:
def get_transformer_embedding(documents):
    output = model(**tokenizer(documents, return_tensors='pt', padding=True))
    return np.mean(output.last_hidden_state.detach().numpy(), axis=1)

In [8]:
poems_embeddings = get_transformer_embedding(df.normal.tolist())

sample_embedding = get_transformer_embedding(['چو ضحاک بشنید اندیشه کرد ز خون پدر شد دلش پر ز د'])
                            
print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(sample_embedding, poems_embeddings):
    print("embedding: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

----------------------------------------------------------------------------------------------------
embedding: "دل روشن من چو برگشت ازوی سوی تخت شاه جهان کرد رو" 	 with similarity of 0.83
embedding: "نپیچد کسی سر ز فرمان اوی نیارد گذشتن ز پیمان اوی" 	 with similarity of 0.83
embedding: "ز خارا گهر جست یک روزگار همی کرد ازو روشنی خواست" 	 with similarity of 0.85
embedding: "چو بیدار گشتم بجستم ز جای چه مایه شب تیره بودم ب" 	 with similarity of 0.85
embedding: "چو ضحاک بشنید اندیشه کرد ز خون پدر شد دلش پر ز د" 	 with similarity of 0.99


In [9]:
MODEL_NAME = "HooshvareLab/bert-base-parsbert-uncased"

config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME, config=config)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
poems_embeddings = get_transformer_embedding(df.normal.tolist())

sample_embedding = get_transformer_embedding(['چو ضحاک بشنید اندیشه کرد ز خون پدر شد دلش پر ز د'])
                            
print('-' * 100)
for poem, sym in get_similars_by_cosine_distance(sample_embedding, poems_embeddings):
    print("embedding: \"{}\" \t with similarity of {:.2f}".format(poem, sym))

----------------------------------------------------------------------------------------------------
embedding: "سخن چون به گوش سیامک رسید ز کردار بدخواه دیو پلی" 	 with similarity of 0.87
embedding: "چو بشنید ازیشان سپهبد سخن یکی نامور نافه افکند ب" 	 with similarity of 0.88
embedding: "ز هرای درندگان چنگ دیو شده سست از خشم کیهان خدیو" 	 with similarity of 0.88
embedding: "چو آگه شد از مرگ فرزند شاه ز تیمار گیتی برو شد س" 	 with similarity of 0.89
embedding: "چو ضحاک بشنید اندیشه کرد ز خون پدر شد دلش پر ز د" 	 with similarity of 1.00
